In [1]:
import pandas as pd
import time

start_time = time.time()

# Replace 'your_file.csv' with the path to your CSV file
file_path = '/Users/bruno/Documents/Statistical_Analyst_Products/combined_ratings.csv'

# Read the CSV file
df = pd.read_csv(file_path)
df['Time'] = pd.to_datetime(df['Review - Time'], unit='s').dt.strftime('%w-%m-%Y')
df['Rating First Digit'] = df['Review - Overall'].astype(str).str[0].astype(int)

# Print the first few rows of the DataFrame
print(df.head())

/var/folders/88/g84kk_9n2498jwvfh8mnxj8w0000gn/T/ipykernel_31703/1582900071.py:10: DtypeWarning: Columns (2,4,6,7,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


   Unnamed: 0                    Name Beer ID  Brewer ID  ABV  \
0           0            Sausa Weizen   47986      10325  5.0   
1           1                Red Moon   48213      10325  6.2   
2           2  Black Horse Black Beer   48215      10325  6.5   
3           3              Sausa Pils   47969      10325  5.0   
4           4           Cauldron DIPA   64883       1075  7.7   

                       Beer Style Review - Appearance Review - Aroma  \
0                      Hefeweizen                 2.5            2.0   
1              English Strong Ale                 3.0            2.5   
2          Foreign / Export Stout                 3.0            2.5   
3                 German Pilsener                 3.5            3.0   
4  American Double / Imperial IPA                 4.0            4.5   

  Review - Palate Review - Taste Review - Overall  Review - Time  \
0             1.5            1.5              1.5     1234817823   
1             3.0            3.0        

In [2]:
print(df.columns)
df2 = df
print(len(df2))

Index(['Unnamed: 0', 'Name', 'Beer ID', 'Brewer ID', 'ABV', 'Beer Style',
       'Review - Appearance', 'Review - Aroma', 'Review - Palate',
       'Review - Taste', 'Review - Overall', 'Review - Time',
       'Review - Profile Name', 'Review - Text', 'File Name', 'Time',
       'Rating First Digit'],
      dtype='object')
4510777


In [3]:
######################################################## ReviewerID
df2 = df2.rename(columns={'ReviewProfileName': 'ReviewerID'})

In [4]:
######################################################## Time
# its already defined

In [5]:
######################################################## ReviewOverall
df2 = df2.rename(columns={'Review - Overall': 'ReviewOverall'})

In [6]:
######################################################## FileName
df2 = df2.rename(columns={'File Name': 'FileName'})

In [7]:
######################################################## BeerRank
# Convert 'Review - Time' to datetime
df2['Review - Time'] = pd.to_datetime(df2['Review - Time'], unit='s')

# Ensure 'ReviewOverall' is treated as numeric, handling non-numeric values as NaN
df2['ReviewOverall'] = pd.to_numeric(df2['ReviewOverall'], errors='coerce')

# Determine the latest date in your dataset
latest_date = df2['Review - Time'].max()

# Calculate the start of the last month period
start_last_month = latest_date - pd.DateOffset(months=1)

# Filter the DataFrame for reviews in the last month
last_month_reviews = df2[(df2['Review - Time'] > start_last_month) & (df2['Review - Time'] <= latest_date)]

# Calculate the average rating for each beer in the last month
beer_rating_last_month = last_month_reviews.groupby('Beer ID')['ReviewOverall'].mean()

# Rank the beers based on this average rating
beer_rank_last_month = beer_rating_last_month.rank(method='dense', ascending=False).rename('BeerRank')

# Reset index to merge correctly
beer_rating_last_month = beer_rating_last_month.reset_index()

# Merge this ranking back into the original DataFrame
df2 = df2.merge(beer_rank_last_month, on='Beer ID', how='left')

In [8]:
######################################################## BeerRating
import pandas as pd

# Convert 'Review - Time' to datetime
df2['Review - Time'] = pd.to_datetime(df2['Review - Time'], unit='s')

# Ensure that 'ReviewOverall' is treated as numeric, with NaN for non-numeric entries
df2['ReviewOverall'] = pd.to_numeric(df2['ReviewOverall'], errors='coerce')

# Determine the latest date in your dataset
latest_date = df2['Review - Time'].max()

# Calculate the start of the last month period
start_last_month = latest_date - pd.DateOffset(months=1)

# Filter the DataFrame for reviews in the last month
last_month_reviews = df2[(df2['Review - Time'] > start_last_month) & (df2['Review - Time'] <= latest_date)]

# Calculate the average rating for each beer in the last month
beer_rating_last_month = last_month_reviews.groupby('Beer ID')['ReviewOverall'].mean()

# Merge this data back into the original DataFrame
df2 = df2.merge(beer_rating_last_month, on='Beer ID', how='left', suffixes=('', '_LastMonth'))

# Renaming the merged column to 'BeerRating'
df2.rename(columns={'ReviewOverall_LastMonth': 'BeerRating'}, inplace=True)

In [9]:
######################################################## BeerABV
df2 = df2.rename(columns={'ABV': 'BeerABV'})

In [10]:
######################################################## BeerPopularity
# Convert 'Review - Time' to datetime if not already done
df2['Review - Time'] = pd.to_datetime(df2['Review - Time'])

# Find the latest date in your dataset, which will act as the 'current' date
current_date = df2['Review - Time'].max()

# Define the date range for the last month
one_month_ago = current_date - pd.DateOffset(months=1)

# Filter reviews in the last month
last_month_reviews = df2[(df2['Review - Time'] > one_month_ago) & (df2['Review - Time'] <= current_date)]

# Count the number of reviews for each beer in the last month
beer_popularity = last_month_reviews.groupby('Beer ID').size().rename('BeerPopularity')

# Join this count back to the original DataFrame
df2 = df2.join(beer_popularity, on='Beer ID', how='left')

# Replace NaN values with 0 - assuming no reviews means 0 popularity
df2['BeerPopularity'].fillna(0, inplace=True)

In [11]:
######################################################## BrewerRank
# Convert 'Review - Time' to datetime if it's not already
df2['Review - Time'] = pd.to_datetime(df2['Review - Time'], unit='s')

# Determine the latest date in your dataset
latest_date = df2['Review - Time'].max()

# Calculate the start of the last month period
start_last_month = latest_date - pd.DateOffset(months=1)

# Filter the DataFrame for reviews in the last month
last_month_reviews = df2[(df2['Review - Time'] > start_last_month) & (df2['Review - Time'] <= latest_date)]

# Calculate the total number of reviews for each brewer in the last month
brewer_reviews_last_month = last_month_reviews.groupby('Brewer ID').size()

# Rank the brewers based on this total number of reviews
# 'dense' ranking method ensures that brewers with the same number of reviews share the same rank
brewer_rank_last_month = brewer_reviews_last_month.rank(method='dense', ascending=False).rename('BrewerRank')

# Merge this ranking back into the original DataFrame
df2 = df2.merge(brewer_rank_last_month, on='Brewer ID', how='left')

In [12]:
######################################################## BrewerPopularity
# Convert 'Review - Time' to datetime if not already done
df2['Review - Time'] = pd.to_datetime(df2['Review - Time'])

# Find the latest date in your dataset, which will act as the 'current' date
current_date = df2['Review - Time'].max()

# Define the date range for the last month
one_month_ago = current_date - pd.DateOffset(months=1)

# Filter reviews in the last month
last_month_reviews = df2[(df2['Review - Time'] > one_month_ago) & (df2['Review - Time'] <= current_date)]

# Count the number of reviews for each beer in the last month
beer_popularity = last_month_reviews.groupby('Brewer ID').size().rename('BrewerPopularity')

# Join this count back to the original DataFrame
df2 = df2.join(beer_popularity, on='Brewer ID', how='left')

# Replace NaN values with 0 - assuming no reviews means 0 popularity
df2['BrewerPopularity'].fillna(0, inplace=True)

In [13]:
######################################################## ReviewNumber
# Sort your DataFrame by 'Beer ID' and 'Review - Time' to ensure that the reviews are in chronological order.
#Use the cumcount() function grouped by 'Beer ID' to assign a sequential number to each review for each beer.

# Ensure your DataFrame is sorted by 'Beer ID' and 'Review - Time'
df2 = df2.sort_values(by=['Beer ID', 'Review - Time'])

# Create the 'ReviewNumber' column
# cumcount() starts from 0, so we add 1 to start the sequence from 1
df2['ReviewNumber'] = df2.groupby('Beer ID').cumcount() + 1

In [14]:
######################################################## ReviewLength
df2['ReviewLength'] = df2['Review - Text'].astype(str).apply(len)

In [15]:
######################################################## CrowdSize
# Convert 'Review - Time' to datetime if it's not already
df2['Review - Time'] = pd.to_datetime(df2['Review - Time'], unit='s')

# Determine the latest date in your dataset
latest_date = df2['Review - Time'].max()

# Calculate the start of the last month period
start_last_month = latest_date - pd.DateOffset(months=1)

# Filter the DataFrame for reviews in the last month
last_month_reviews = df2[(df2['Review - Time'] > start_last_month) & (df2['Review - Time'] <= latest_date)]

# Calculate the total number of reviews for each beer style/category in the last month
crowd_size_last_month = last_month_reviews.groupby('Beer Style').size().rename('CrowdSize')

# Merge this count back into the original DataFrame
df2 = df2.merge(crowd_size_last_month, on='Beer Style', how='left')


In [16]:
######################################################## ReviewerTenure
# Convert 'Review - Time' from Unix timestamp to datetime
df2['Review - Time'] = pd.to_datetime(df2['Review - Time'], unit='s')

# Calculate the earliest review date for each reviewer
# Replace 'ReviewerID' with the column that uniquely identifies each reviewer in your DataFrame
earliest_review = df2.groupby('Review - Profile Name')['Review - Time'].min()

# Map the earliest review date to each row in the DataFrame
df2['EarliestReview'] = df2['Review - Profile Name'].map(earliest_review)

# Calculate the tenure as the difference between the review date and the earliest review date
df2['ReviewerTenure'] = df2['Review - Time'] - df2['EarliestReview']

# Convert tenure to the number of days
df2['ReviewerTenure'] = df2['ReviewerTenure'].dt.days

In [17]:
######################################################## ReviewerExperience
# Ensure your DataFrame is sorted by reviewer ID and review time
df2 = df2.sort_values(by=['Review - Profile Name', 'Review - Time'])

# Create the 'ReviewerExperience' column
# cumcount() starts from 0, so we add 1 to start the count from 1
df2['ReviewerExperience'] = df2.groupby('Review - Profile Name').cumcount() + 1

**TABLE 1**

In [27]:
import pandas as pd
from scipy import stats

# List of variables for the analysis
variables = ['ReviewOverall', 'BeerRank', 'BeerRating', 'BeerPopularity', 'BrewerRank', 
             'BrewerPopularity', 'ReviewNumber', 'ReviewLength', 'CrowdSize', 'ReviewerTenure', 'ReviewerExperience']

# Splitting the dataset specifically for 'beeradvocate' and 'ratebeer'
df_platform1 = df2[df2['FileName'] == 'beeradvocate']
df_platform2 = df2[df2['FileName'] == 'ratebeer']

# Prepare a list to collect rows for Table 1
table1_data = []

# Total number of variables
total_variables = len(variables)
current_variable = 0

# Iterate over each variable to calculate and print the statistics
for var in variables:
    current_variable += 1
    print(f"Processing variable {current_variable} of {total_variables}: {var}")

    # Calculating means and standard deviations
    mean1 = df_platform1[var].mean()
    sd1 = df_platform1[var].std()
    mean2 = df_platform2[var].mean()
    sd2 = df_platform2[var].std()
    print("  Calculated means and standard deviations.")

    # Perform t-test
    t_test_result = stats.ttest_ind(df_platform1[var].dropna(), df_platform2[var].dropna(), equal_var=False)
    print(f"  Performed t-test. T-statistic: {t_test_result.statistic:.10f}")

    # Add the results to the table data
    row = {
        'Variable': var, 
        'Mean_beeradvocate': mean1, 
        'SD_beeradvocate': sd1, 
        'Mean_ratebeer': mean2, 
        'SD_ratebeer': sd2, 
        'T-Statistic': f"{t_test_result.statistic:.10f}"
    }
    table1_data.append(row)
    print("  Finished processing.")

# Create DataFrame from the list
table1 = pd.DataFrame(table1_data)

# Displaying the final Table 1
print("\nCompleted all calculations. Displaying Table 1:")
table1


Processing variable 1 of 11: ReviewOverall
  Calculated means and standard deviations.
  Performed t-test. T-statistic: nan
  Finished processing.
Processing variable 2 of 11: BeerRank
  Calculated means and standard deviations.
  Performed t-test. T-statistic: -153.0229733913
  Finished processing.
Processing variable 3 of 11: BeerRating
  Calculated means and standard deviations.
  Performed t-test. T-statistic: 140.4502185641
  Finished processing.
Processing variable 4 of 11: BeerPopularity
  Calculated means and standard deviations.
  Performed t-test. T-statistic: 220.6620681993
  Finished processing.
Processing variable 5 of 11: BrewerRank
  Calculated means and standard deviations.
  Performed t-test. T-statistic: -193.5249334958
  Finished processing.
Processing variable 6 of 11: BrewerPopularity
  Calculated means and standard deviations.
  Performed t-test. T-statistic: 207.2489633609
  Finished processing.
Processing variable 7 of 11: ReviewNumber
  Calculated means and sta

,Variable,Mean_beeradvocate,SD_beeradvocate,Mean_ratebeer,SD_ratebeer,T-Statistic
0,ReviewOverall,3.815581,0.720622,NaN,NaN,nan
1,BeerRank,175.214059,93.006202,204.668358,93.430071,-153.0229733913
2,BeerRating,3.792460,0.543009,3.612916,0.639247,140.4502185641
3,BeerPopularity,7.011506,11.122742,4.820897,7.757442,220.6620681993
4,BrewerRank,105.500490,68.370963,118.736237,65.971992,-193.5249334958
5,BrewerPopularity,146.672266,173.496381,113.014094,147.135028,207.2489633609
6,ReviewNumber,319.570206,472.616903,266.134632,430.192288,118.2879518057
7,ReviewLength,695.010994,377.965479,307.886899,215.019847,1189.8824761055
8,CrowdSize,658.057147,593.740888,991.607643,745.857354,-519.3534361655
9,ReviewerTenure,598.978528,639.005760,864.721190,796.986112,-385.7282613325


Why are we getting Nan for Review Overall? It seems that we dont have ratings as we can see in the following snippet:

In [25]:
# Filter for 'ratebeer'
df_ratebeer = df2[df2['FileName'] == 'ratebeer']

# Calculate and print the maximum, minimum, and mean of 'ReviewOverall' for 'ratebeer'
max_review_overall = df_ratebeer['ReviewOverall'].max()
min_review_overall = df_ratebeer['ReviewOverall'].min()
mean_review_overall = df_ratebeer['ReviewOverall'].mean()

print("Maximum of 'ReviewOverall' for ratebeer:", max_review_overall)
print("Minimum of 'ReviewOverall' for ratebeer:", min_review_overall)
print("Mean of 'ReviewOverall' for ratebeer:", mean_review_overall)

Maximum of 'ReviewOverall' for ratebeer: nan
Minimum of 'ReviewOverall' for ratebeer: nan
Mean of 'ReviewOverall' for ratebeer: nan


**TABLE 2**

In [30]:
import statsmodels.api as sm
import pandas as pd

# Assuming df2 is your DataFrame
# Replace df2 with the actual DataFrame you are working with

# Check and convert data types if necessary
# Ensure all control variables are numeric
for column in ['BeerRank', 'BeerABV', 'BeerPopularity', 'BrewerRank', 
               'BrewerPopularity', 'ReviewNumber', 'ReviewLength', 
               'CrowdSize', 'ReviewerTenure', 'ReviewerExperience']:
    df2[column] = pd.to_numeric(df2[column], errors='coerce')

# Handling missing values
# You can choose to fill missing values or drop them. Here, we'll drop them
df2 = df2.dropna(subset=['BeerRating', 'BeerRank', 'BeerABV', 'BeerPopularity', 
                         'BrewerRank', 'BrewerPopularity', 'ReviewNumber', 
                         'ReviewLength', 'CrowdSize', 'ReviewerTenure', 
                         'ReviewerExperience'])

# Define the dependent variable
y = df2['BeerRating']

# Define the independent variables (control variables in this case)
X = df2[['BeerRank', 'BeerABV', 'BeerPopularity', 'BrewerRank', 
         'BrewerPopularity', 'ReviewNumber', 'ReviewLength', 
         'CrowdSize', 'ReviewerTenure', 'ReviewerExperience']]

# Add a constant to the model (intercept)
X = sm.add_constant(X)

# Fit the regression model
model = sm.OLS(y, X).fit()

# Print out the regression results
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:             BeerRating   R-squared:                       0.842
Model:                            OLS   Adj. R-squared:                  0.842
Method:                 Least Squares   F-statistic:                 7.316e+05
Date:                Thu, 11 Jan 2024   Prob (F-statistic):               0.00
Time:                        23:29:41   Log-Likelihood:                 96610.
No. Observations:             1371248   AIC:                        -1.932e+05
Df Residuals:                 1371237   BIC:                        -1.931e+05
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  4.7817      0